In [27]:
!python3 -m pip install --user sklearn

    100% |████████████████████████████████| 5.3MB 253kB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/ab/19/c0ad5b9183ef97030edd6297d1726525ff2c369a09fbb6ea52a1e616ffd6/scipy-1.2.0-cp35-cp35m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/86/04/bd774106ae0ae1ada68c67efe89f1a16b2aa373cc2db15d974002a9f136d/numpy-1.15.4-cp35-cp35m-manylinux1_x86_64.whl
  Running setup.py bdist_wheel for sklearn ... done
  Stored in directory: /home/kitto727/.cache/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn


In [2]:
import h5py
import tensorflow as tf
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# %matplotlib inline

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Reshape, Concatenate, Add
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D, BatchNormalization
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras.utils import Sequence, to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import save_model, load_model
from tensorflow.keras.optimizers import Adam, RMSprop

import os, sys, re, codecs

In [3]:
TPU_SERVER = '10.35.75.2'
TPU_PORT = ':8470'
TPU_WORKER = 'grpc://' + TPU_SERVER + TPU_PORT
TPU_WORKER

'grpc://10.35.75.2:8470'

In [4]:
def generate_model(max_features, max_len, embed_size, tg_cate_size):
    inputs = []
    embeddings = []

    inp_product  = Input(shape=(max_len, ))
    embedding = Embedding(max_features, embed_size, input_length=max_len)(inp_product)
    embedding = Reshape(target_shape=(max_len*embed_size,))(embedding)
    embedding = Dense(256, activation='relu')(embedding)
    inputs.append(inp_product)
    embeddings.append(embedding)

    inp_model  = Input(shape=(max_len, ))
    embedding = Embedding(max_features, embed_size, input_length=max_len)(inp_model)
    embedding = Reshape(target_shape=(max_len*embed_size,))(embedding)
    embedding = Dense(256, activation='relu')(embedding)
    inputs.append(inp_model)
    embeddings.append(embedding)

    inp_brand  = Input(shape=(max_len, ))
    embedding = Embedding(max_features, embed_size, input_length=max_len)(inp_brand)
    embedding = Reshape(target_shape=(max_len*embed_size,))(embedding)
    embedding = Dense(256, activation='relu')(embedding)
    inputs.append(inp_brand)
    embeddings.append(embedding)

    inp_maker  = Input(shape=(max_len, ))
    embedding = Embedding(max_features, embed_size, input_length=max_len)(inp_maker)
    embedding = Reshape(target_shape=(max_len*embed_size,))(embedding)
    embedding = Dense(256, activation='relu')(embedding)
    inputs.append(inp_maker)
    embeddings.append(embedding)

    inp_img = Input(shape=(2048, ))
    feat = Dropout(0.4)(inp_img)
    feat = Dense(256, activation='relu')(feat)
    inputs.append(inp_img)
    embeddings.append(feat)

    x = Concatenate()(embeddings)
    x = Dropout(0.3)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(256, activation='relu')(x)

    # decoder1 = add([x, feat])
    # decoder2 = Dense(256, activation='relu')(decoder1)
    output = Dense(tg_cate_size, activation='softmax')(x)

    model = Model(inputs, output)
    # model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

    model.summary()
    
    return model

In [4]:
!free

              total        used        free      shared  buff/cache   available
Mem:      165060836      601468   101977076         868    62482292   162947068
Swap:             0           0           0


In [5]:
def data_generator(list_X, y, batch_size, total_size):
    start_idx = 0
    end_idx = 0
    
    while True:
        if (start_idx + batch_size) >= total_size:
            start_idx = 0
            end_idx = 0
        
        end_idx += batch_size
        batch_X = []
        for X in list_X:
            batch_X.append(X[start_idx:end_idx])
            
        batch_y = np.array(y[start_idx:end_idx])
        start_idx = end_idx        
        
        yield batch_X, batch_y

In [6]:
X_product = h5py.File('origin/X_train_product.h5py', 'r')
X_maker = h5py.File('origin/X_train_maker.h5py', 'r')
X_brand = h5py.File('origin/X_train_brand.h5py', 'r')
X_model = h5py.File('origin/X_train_model.h5py', 'r')
X_image = h5py.File('origin/X_train_img_feat.h5py', 'r')

X_job_product = h5py.File('bcate/X_train_product_for_b.h5py', 'r')
X_job_maker = h5py.File('bcate/X_train_maker_for_b.h5py', 'r')
X_job_brand = h5py.File('bcate/X_train_brand_for_b.h5py', 'r')
X_job_model = h5py.File('bcate/X_train_model_for_b.h5py', 'r')
X_job_image = h5py.File('bcate/X_img_for_b.h5py', 'r')

y_b_cate = h5py.File('origin/y_train_bcateid.h5py', 'r')
y_m_cate = h5py.File('origin/y_train_mcateid.h5py', 'r')
y_s_cate = h5py.File('origin/y_train_scateid.h5py', 'r')
y_d_cate = h5py.File('origin/y_train_dcateid.h5py', 'r')

y_b_job = h5py.File('bcate/y_bcateid_for_b.h5py', 'r')

print("Product / train : {}, val : {}".format(X_product['train'].shape, X_product['val'].shape))
print("Maker / train : {}, val : {}".format(X_maker['train'].shape, X_maker['val'].shape))
print("Brand / train : {}, val : {}".format(X_brand['train'].shape, X_brand['val'].shape))
print("Model / train : {}, val : {}".format(X_model['train'].shape, X_model['val'].shape))
print("Image / train : {}, val : {}".format(X_image['train'].shape, X_image['val'].shape))

print("job_Product / train : {}".format(X_job_product['train'].shape))
print("job_Maker / train : {}".format(X_job_maker['train'].shape))
print("job_Brand / train : {}".format(X_job_brand['train'].shape))
print("job_Model / train : {}".format(X_job_model['train'].shape))
print("job_Image / train : {}".format(X_job_image['train'].shape))

print("B cate / train : {}, val : {}".format(y_b_cate['train'].shape, y_b_cate['val'].shape))
print("M cate / train : {}, val : {}".format(y_m_cate['train'].shape, y_m_cate['val'].shape))
print("S cate / train : {}, val : {}".format(y_s_cate['train'].shape, y_s_cate['val'].shape))
print("D cate / train : {}, val : {}".format(y_d_cate['train'].shape, y_d_cate['val'].shape))

print("D cate / train : {}".format(y_b_job['train'].shape))

Product / train : (6507854, 32), val : (1626964, 32)
Maker / train : (6507854, 32), val : (1626964, 32)
Brand / train : (6507854, 32), val : (1626964, 32)
Model / train : (6507854, 32), val : (1626964, 32)
Image / train : (6507854, 2048), val : (1626964, 2048)
job_Product / train : (869557, 32)
job_Maker / train : (869557, 32)
job_Brand / train : (869557, 32)
job_Model / train : (869557, 32)
job_Image / train : (869557, 2048)
B cate / train : (6507854, 57), val : (1626964, 57)
M cate / train : (6507854, 552), val : (1626964, 552)
S cate / train : (6507854, 3191), val : (1626964, 3191)
D cate / train : (6507854, 405), val : (1626964, 405)
D cate / train : (869557, 57)


In [79]:
X_product_s = h5py.File('./Taejin/X_product_for_s.h5py', 'r')
X_maker_s = h5py.File('./Taejin/X_maker_for_s.h5py', 'r')
X_brand_s = h5py.File('./Taejin/X_brand_for_s.h5py', 'r')
X_model_s = h5py.File('./Taejin/X_model_for_s.h5py', 'r')
X_image_s = h5py.File('./Taejin/X_image_for_s.h5py', 'r')

X_product_d = h5py.File('./Taejin/X_product_for_d.h5py', 'r')
X_maker_d = h5py.File('./Taejin/X_maker_for_d.h5py', 'r')
X_brand_d = h5py.File('./Taejin/X_brand_for_d.h5py', 'r')
X_model_d = h5py.File('./Taejin/X_model_for_d.h5py', 'r')
X_image_d = h5py.File('./Taejin/X_image_for_d.h5py', 'r')

y_b_cate_s = h5py.File('./Taejin/y_bcateid_rm_for_s.h5py', 'r')
y_m_cate_s = h5py.File('./Taejin/y_mcateid_rm_for_s.h5py', 'r')
y_s_cate_s = h5py.File('./Taejin/y_scateid_rm.h5py', 'r')

y_b_cate_d = h5py.File('./Taejin/y_bcateid_rm_for_d.h5py', 'r')
y_m_cate_d = h5py.File('./Taejin/y_mcateid_rm_for_d.h5py', 'r')
y_s_cate_d = h5py.File('./Taejin/y_scateid_rm_for_d.h5py', 'r')
y_d_cate_d = h5py.File('./Taejin/y_dcateid_rm.h5py', 'r')

print("Product for s / train : {}, val : {}".format(X_product_s['train'].shape, X_product_s['val'].shape))
print("Maker for s / train : {}, val : {}".format(X_maker_s['train'].shape, X_maker_s['val'].shape))
print("Brand for s / train : {}, val : {}".format(X_brand_s['train'].shape, X_brand_s['val'].shape))
print("Model for s / train : {}, val : {}".format(X_model_s['train'].shape, X_model_s['val'].shape))
print("Image for s / train : {}, val : {}".format(X_image_s['train'].shape, X_image_s['val'].shape))

print("Product for d / train : {}, val : {}".format(X_product_d['train'].shape, X_product_d['val'].shape))
print("Maker for d / train : {}, val : {}".format(X_maker_d['train'].shape, X_maker_d['val'].shape))
print("Brand for d / train : {}, val : {}".format(X_brand_d['train'].shape, X_brand_d['val'].shape))
print("Model for d / train : {}, val : {}".format(X_model_d['train'].shape, X_model_d['val'].shape))
print("Image for d / train : {}, val : {}".format(X_image_d['train'].shape, X_image_d['val'].shape))

print("b cate for s / {} {}".format(y_b_cate_s['train'].shape, y_b_cate_s['val'].shape))
print("m cat for d / {} {}".format(y_m_cate_s['train'].shape, y_m_cate_s['val'].shape))
print("s cate for s / {} {}".format(y_s_cate_s['train'].shape, y_s_cate_s['val'].shape))
print("d cat for d / {} {}".format(y_d_cate_d['train'].shape, y_d_cate_d['val'].shape))

Product for s / train : (5055885, 32), val : (1212813, 32)
Maker for s / train : (5055885, 32), val : (1212813, 32)
Brand for s / train : (5055885, 32), val : (1212813, 32)
Model for s / train : (5055885, 32), val : (1212813, 32)
Image for s / train : (5055885, 2048), val : (1212813, 2048)
Product for d / train : (607560, 32), val : (150332, 32)
Maker for d / train : (607560, 32), val : (150332, 32)
Brand for d / train : (607560, 32), val : (150332, 32)
Model for d / train : (607560, 32), val : (150332, 32)
Image for d / train : (607560, 2048), val : (150332, 2048)
b cate for s / (5055885, 57) (1212813, 57)
m cat for d / (5055885, 552) (1212813, 552)
s cate for s / (5055885, 3191) (1212813, 3191)
d cat for d / (607560, 405) (150332, 405)


In [7]:
max_features = 100000
max_len = 32
embed_size = 128
cate_size = 3191

In [30]:
!free

              total        used        free      shared  buff/cache   available
Mem:      165060836     7608712    79500524         868    77951600   155939796
Swap:             0           0           0


In [8]:
batch_size = 4096*8

In [9]:
model = generate_model(max_features, max_len, embed_size, cate_size)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 32)           0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 32)           0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 32)           0                                            
__________________________________________________________________________________________________
embedding 

In [10]:
tf.keras.backend.clear_session()

tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))

INFO:tensorflow:Querying Tensorflow master (b'grpc://10.35.75.2:8470') for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 8977423443693438216)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 15150836544296562072)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 1801436362513015490)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 6890304418270238500)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 10778465270698302976)
INFO:tensorflow:*** Available Device: _DeviceAt

In [34]:
# tpu_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
tpu_model.compile(loss='categorical_crossentropy', optimizer=tf.train.AdamOptimizer(), metrics=['accuracy'])

In [ ]:
# train_generator = data_generator([X_product['train'], X_model['train'], X_brand['train'], X_maker['train'], X_image['train']], y_m_cate['train'], batch_size, len(X_product['train']))
# valid_generator = data_generator([X_product['val'], X_model['val'], X_brand['val'], X_maker['val'],   X_image['val']], y_m_cate['val'], batch_size, len(X_product['val']))

In [23]:
# train_generator = data_generator([X_product_d['train'], X_model_d['train'], X_brand_d['train'], X_maker_d['train'], X_image_d['train']], y_d_cate_d['train'], batch_size, len(X_product_d['train']))
# valid_generator = data_generator([X_product_d['val'], X_model_d['val'], X_brand_d['val'], X_maker_d['val'], X_image_d['val']], y_d_cate_d['val'], batch_size, len(X_product_d['val']))

In [35]:
train_generator = data_generator([X_product_s['train'], X_model_s['train'], X_brand_s['train'], X_maker_s['train'], X_image_s['train']], y_s_cate_s['train'], batch_size, len(X_product_s['train']))
valid_generator = data_generator([X_product_s['val'], X_model_s['val'], X_brand_s['val'], X_maker_s['val'], X_image_s['val']], y_s_cate_s['val'], batch_size, len(X_product_s['val']))

In [36]:
# early_stopping = EarlyStopping(monitor='acc', mode = 'max',patience=5, verbose=1)

model_checkpoint = ModelCheckpoint('./Taejin/s_model_rm.h5',monitor='acc', 
                                   mode = 'max', save_best_only=True, verbose=1)
# # reduce_lr = ReduceLROnPlateau(monitor='acc', mode = 'max',factor=0.5, patience=5, min_lr=0.0001, verbose=1)

In [ ]:
tpu_model.fit_generator(generator=train_generator, 
                        steps_per_epoch=int(len(X_product_s['train'])/batch_size)+1, 
                        validation_data=valid_generator, 
                        validation_steps=int(len(X_product_s['val'])/batch_size)+1, epochs=20,
                        callbacks=[model_checkpoint])

# Job training

In [11]:
load_b = load_model('b_model_2.h5')

In [27]:
load_b.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 32)           0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 32)           0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 32)           0                                            
__________________________________________________________________________________________________
embedding 

In [12]:
# tpu_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
load_b.compile(loss='categorical_crossentropy', optimizer=tf.train.AdamOptimizer(), metrics=['accuracy'])

In [13]:
batch_size = 1024*8

In [1]:
print(batch_size)

NameError: name 'batch_size' is not defined

In [14]:
np_product = np.array(X_job_product['train'])
np_maker = np.array(X_job_maker['train'])
np_brand = np.array(X_job_brand['train'])
np_model = np.array(X_job_model['train'])
np_image = np.array(X_job_image['train'])

In [15]:
np_y_job = np.array(y_b_job['train'])

In [16]:
from sklearn.model_selection import train_test_split

job_product_train, job_product_val, job_maker_train, job_maker_val, job_brand_train, job_brand_val, job_model_train, job_model_val, job_image_train, job_image_val, y_b_job_train, y_b_job_val = train_test_split(np_product, np_maker, np_brand, np_model, np_image, np_y_job, 0.2)

TypeError: Singleton array array(0.2) cannot be considered a valid collection.

In [ ]:
train_generator = data_generator([X_job_product['train'], X_job_model['train'], X_job_brand['train'], X_job_maker['train'], X_job_image['train']], y_b_job['train'], batch_size, len(X_job_product['train']))
valid_generator = data_generator([X_product['val'], X_model['val'], X_brand['val'], X_maker['val'], X_image['val']], y_b_cate['val'], batch_size, len(X_product['val']))

In [ ]:
# early_stopping = EarlyStopping(monitor='acc', mode = 'max',patience=5, verbose=1)

model_checkpoint = ModelCheckpoint('b_model_job.h5',monitor='acc', 
                                   mode = 'max', save_best_only=True, verbose=1)
# # reduce_lr = ReduceLROnPlateau(monitor='acc', mode = 'max',factor=0.5, patience=5, min_lr=0.0001, verbose=1)

In [ ]:
load_b.fit_generator(generator=train_generator, 
                        steps_per_epoch=int(len(X_job_product['train'])/batch_size)+1, 
                        validation_data=valid_generator, 
                        validation_steps=int(len(X_product['val'])/batch_size)+1, epochs=20,
                        callbacks=[model_checkpoint])

# M model training!

In [ ]:
for i in load_b.layers:
    print(i.input)

In [ ]:
load_b.layers[-6].output

In [ ]:
load_b = load_model('tpu_b_model.h5')

In [ ]:
load_b.trainable = False

In [ ]:
new_m_input = Input(shape=(1,), name='m_input')
new_m = Dense(256, activation='relu', name='dense_1_m1')(new_m_input)
concate_m = Concatenate(name='concate_m1')([load_b.layers[-6].output, new_m])
# new_m = BatchNormalization()(concate_m)
new_m = Dense(512, activation='relu', name='dense_2_m1')(concate_m)
new_m = Dropout(0.2, name='drop_2')(new_m)
new_m = Dense(256, activation='relu', name='dense_4_m1')(new_m)
new_m = Dropout(0.2, name='drop_3')(new_m)
output = Dense(552, activation='softmax', name='dense_3_m1')(new_m)

model_m = Model([load_b.layers[4].input,load_b.layers[5].input,load_b.layers[6].input,load_b.layers[7].input,load_b.layers[8].input,  new_m_input], output)

In [ ]:
def data_generator_m(list_X,input_cate, y, batch_size, total_size):
    start_idx = 0
    end_idx = 0
    while True:
        if (start_idx + batch_size) >= total_size:
            start_idx = 0
            end_idx = 0
            
        end_idx += batch_size
#         offset = np.random.randint(0, total_size - batch_size)
        batch_X = []
        for X in list_X:
            batch_X.append(np.array(X[start_idx:end_idx]))
            
        cate_input = np.argmax(input_cate[start_idx:end_idx], axis=1)+1
        batch_X.append(cate_input)    
        batch_y = np.array(y[start_idx:end_idx])
        start_idx = end_idx        
        
        yield batch_X, batch_y

In [ ]:
train_generator_m = data_generator_m([X_product['train'], X_model['train'], X_brand['train'], X_maker['train'], X_image['train']],y_b_cate['train'], y_m_cate['train'], batch_size, len(X_product['train']))
valid_generator_m = data_generator_m([X_product['val'], X_model['val'], X_brand['val'], X_maker['val'],   X_image['val']],y_b_cate['val'], y_m_cate['val'], batch_size, len(X_product['val']))

In [ ]:
next(train_generator_m)[0][-1][20]

In [ ]:
model_m.compile(loss='categorical_crossentropy', optimizer=tf.train.AdamOptimizer(), metrics=['acc'])
# early_stopping = EarlyStopping(monitor='acc', mode = 'max',patience=5, verbose=1)

ft_checkpoint = ModelCheckpoint('m_model_transfer.h5',monitor='acc', 
                                   mode = 'max', save_best_only=True, verbose=1)
# # reduce_lr = ReduceLROnPlateau(monitor='acc', mode = 'max',factor=0.5, patience=5, min_lr=0.0001, verbose=1)

In [ ]:
model_m.fit_generator(generator=train_generator_m, 
                        steps_per_epoch=int(6507854/batch_size)+1, 
                        validation_data=valid_generator_m, 
                        validation_steps=int(1626964/batch_size)+1, epochs=10,
                        callbacks=[ft_checkpoint])

In [ ]:
model_m.save('tpu_m_transfer.h5')

# S_transfer model test!

In [47]:
load_m = load_model('m_model_transfer.h5')
load_m.trainable = False

In [48]:
print(load_m.layers[0].input)
print(load_m.layers[1].input)
print(load_m.layers[2].input)
print(load_m.layers[3].input)
print(load_m.layers[8].input)
print(load_m.layers[-9].input)
print(load_m.layers[-6].output)

Tensor("input_1_1:0", shape=(?, 32), dtype=float32)
Tensor("input_2_1:0", shape=(?, 32), dtype=float32)
Tensor("input_3_1:0", shape=(?, 32), dtype=float32)
Tensor("input_4_1:0", shape=(?, 32), dtype=float32)
Tensor("input_5_1:0", shape=(?, 2048), dtype=float32)
Tensor("m_input:0", shape=(?, 1), dtype=float32)
Tensor("concate_m1/concat:0", shape=(?, 1536), dtype=float32)


In [51]:
new_s = Input(shape=(1,), name='input_s')
new1 = Dense(256, activation='relu')(new_s)
concate_m = Concatenate()([load_m.layers[-6].output, new1])
# new1 = BatchNormalization()(concate_m)
new1 = Dense(256, activation='relu')(concate_m)
new1 = Dropout(0.2)(new1)
output = Dense(3191, activation='softmax')(new1)

model_s = Model([load_m.layers[0].input,load_m.layers[1].input,load_m.layers[2].input,load_m.layers[3].input,load_m.layers[8].input, load_m.layers[-9].input, new_s], output)

In [52]:
max_features = 100000
max_len = 32
embed_size = 128
cate_size = 3191

In [66]:
def data_generator_s(list_X,input_cate_b, input_cate_m, y, batch_size, total_size):
#     start_idx = 0
#     end_idx = 0
    while True:
#         end_idx += batch_size
        offset = np.random.randint(0, total_size - batch_size)
        batch_X = []
        for X in list_X:
            batch_X.append(np.array(X[offset:offset+batch_size]))
            
        cate_input_b = np.argmax(input_cate_b[offset:offset+batch_size], axis=1)+1
        batch_X.append(cate_input_b)
        cate_input_m = np.argmax(input_cate_m[offset:offset+batch_size], axis=1)+1
        batch_X.append(cate_input_m)
        batch_y = np.array(y[offset:offset+batch_size])
#         start_idx = end_idx        
        
        yield batch_X, batch_y

In [ ]:
y_b_cate_s = h5py.File('./Taejin/y_bcateid_rm_for_s.h5py', 'r')
y_m_cate_s = h5py.File('./Taejin/y_mcateid_rm_for_s.h5py', 'r')
y_s_cate_s = h5py.File('./Taejin/y_scateid_rm.h5py', 'r')

In [70]:
train_generator_s = data_generator_s([X_product_s['train'], X_model_s['train'], X_brand_s['train'], X_maker_s['train'], X_image_s['train']],y_b_cate_s['train'], y_m_cate_s['train'],y_s_cate_s['train'], batch_size, len(X_product_s['train']))
valid_generator_s = data_generator_s([X_product_s['val'], X_model_s['val'], X_brand_s['val'], X_maker_s['val'], X_image_s['val']],y_b_cate_s['val'], y_m_cate_s['val'], y_s_cate_s['val'], batch_size, len(X_product_s['val']))

In [71]:
model_s.compile(loss='categorical_crossentropy', optimizer=tf.train.AdamOptimizer(), metrics=['accuracy'])
# early_stopping = EarlyStopping(monitor='acc', mode = 'max',patience=5, verbose=1)

s_checkpoint = ModelCheckpoint('s_model_rm.h5',monitor='acc', 
                                   mode = 'max', save_best_only=True, verbose=1)
# # reduce_lr = ReduceLROnPlateau(monitor='acc', mode = 'max',factor=0.5, patience=5, min_lr=0.0001, verbose=1)

In [73]:
model_s.fit_generator(generator=train_generator_s, 
                        steps_per_epoch=int(len(X_product_s['train'])/batch_size)+1, 
                        validation_data=valid_generator_s, 
                        validation_steps=int(len(X_product_s['val'])/batch_size)+1, epochs=10,
                        callbacks=[s_checkpoint])

Epoch 1/10
154/155 [============================>.] - ETA: 4s - loss: 1.9674 - acc: 0.5543
Epoch 00001: acc improved from -inf to 0.55557, saving model to s_model_rm.h5
155/155 [==============================] - 801s 5s/step - loss: 1.9607 - acc: 0.5556 - val_loss: 0.7775 - val_acc: 0.7934
Epoch 2/10
154/155 [============================>.] - ETA: 4s - loss: 0.6921 - acc: 0.8079
Epoch 00002: acc improved from 0.55557 to 0.80795, saving model to s_model_rm.h5
155/155 [==============================] - 818s 5s/step - loss: 0.6916 - acc: 0.8079 - val_loss: 0.6123 - val_acc: 0.8349
Epoch 3/10
154/155 [============================>.] - ETA: 4s - loss: 0.5194 - acc: 0.8529
Epoch 00003: acc improved from 0.80795 to 0.85295, saving model to s_model_rm.h5
155/155 [==============================] - 861s 6s/step - loss: 0.5195 - acc: 0.8529 - val_loss: 0.5296 - val_acc: 0.8564
Epoch 4/10
154/155 [============================>.] - ETA: 4s - loss: 0.4371 - acc: 0.8751
Epoch 00004: acc improved from

KeyboardInterrupt: 

# D model test!

In [74]:
load_d = load_model('s_model_rm.h5')
load_d.trainable = False

In [ ]:
for i in load_d.layers:
    print(i.input)

In [75]:
print(load_d.layers[4].input)
print(load_d.layers[5].input)
print(load_d.layers[6].input)
print(load_d.layers[7].input)
print(load_d.layers[8].input)
print(load_d.layers[-10].input)
print(load_d.layers[-7].input)
print(load_d.layers[-4].output)

Tensor("input_1_2:0", shape=(?, 32), dtype=float32)
Tensor("input_2_2:0", shape=(?, 32), dtype=float32)
Tensor("input_3_2:0", shape=(?, 32), dtype=float32)
Tensor("input_4_2:0", shape=(?, 32), dtype=float32)
Tensor("input_5_2:0", shape=(?, 2048), dtype=float32)
Tensor("m_input_1:0", shape=(?, 1), dtype=float32)
Tensor("input_s_3:0", shape=(?, 1), dtype=float32)
Tensor("concatenate_2_2/concat:0", shape=(?, 1792), dtype=float32)


In [76]:
new_d = Input(shape=(1,), name='input_d')
new1 = Dense(256, activation='relu')(new_d)
concate_d = Concatenate()([load_d.layers[-4].output, new1])
# new1 = BatchNormalization()(concate_m)
new1 = Dense(256, activation='relu')(concate_d)
new1 = Dropout(0.2)(new1)
output = Dense(405, activation='softmax')(new1)

model_d = Model([load_d.layers[4].input,load_d.layers[5].input,load_d.layers[6].input,load_d.layers[7].input,load_d.layers[8].input, load_d.layers[-10].input, load_d.layers[-7].input, new_d], output)

In [77]:
def data_generator_d(list_X,input_cate_b, input_cate_m, input_cate_s, y, batch_size, total_size):
    start_idx = 0
    end_idx = 0
    while True:
        if (start_idx + batch_size) >= total_size:
            start_idx = 0
            end_idx = 0
        end_idx += batch_size
#         offset = np.random.randint(0, total_size - batch_size)
        batch_X = []
        for X in list_X:
            batch_X.append(np.array(X[start_idx:end_idx]))
            
        cate_input_b = np.argmax(input_cate_b[start_idx:end_idx], axis=1)+1
        batch_X.append(cate_input_b)
        cate_input_m = np.argmax(input_cate_m[start_idx:end_idx], axis=1)+1
        batch_X.append(cate_input_m)
        cate_input_s = np.argmax(input_cate_s[start_idx:end_idx], axis=1)
        batch_X.append(cate_input_s)
        batch_y = np.array(y[start_idx:end_idx])
        start_idx = end_idx        
        
        yield batch_X, batch_y

In [80]:
batch_size=512*8

In [81]:
train_generator_d = data_generator_d([X_product_d['train'], X_model_d['train'], X_brand_d['train'], X_maker_d['train'], X_image_d['train']],y_b_cate_d['train'], y_m_cate_d['train'],y_s_cate_d['train'], y_d_cate_d['train'], batch_size, len(X_product_d['train']))
valid_generator_d = data_generator_d([X_product_d['val'], X_model_d['val'], X_brand_d['val'], X_maker_d['val'], X_image_d['val']],y_b_cate_d['val'], y_m_cate_d['val'], y_s_cate_d['val'], y_d_cate_d['val'], batch_size, len(X_product_d['val']))

In [82]:
model_d.compile(loss='categorical_crossentropy', optimizer=tf.train.AdamOptimizer(), metrics=['accuracy'])
# early_stopping = EarlyStopping(monitor='acc', mode = 'max',patience=5, verbose=1)

d_checkpoint = ModelCheckpoint('d_model_rm.h5',monitor='acc', 
                                   mode = 'max', save_best_only=True, verbose=1)
# # reduce_lr = ReduceLROnPlateau(monitor='acc', mode = 'max',factor=0.5, patience=5, min_lr=0.0001, verbose=1)

In [83]:
model_d.fit_generator(generator=train_generator_d, 
                        steps_per_epoch=int(len(X_product_d['train'])/batch_size)+1, 
                        validation_data=valid_generator_d, 
                        validation_steps=int(len(X_product_d['val'])/batch_size)+1, epochs=10,
                        callbacks=[d_checkpoint])

Epoch 1/10
148/149 [============================>.] - ETA: 0s - loss: 2.6140 - acc: 0.5301
Epoch 00001: acc improved from -inf to 0.53142, saving model to d_model_rm.h5
149/149 [==============================] - 59s 395ms/step - loss: 2.6036 - acc: 0.5314 - val_loss: 1.5718 - val_acc: 0.6585
Epoch 2/10
148/149 [============================>.] - ETA: 0s - loss: 1.3651 - acc: 0.6809
Epoch 00002: acc improved from 0.53142 to 0.68199, saving model to d_model_rm.h5
149/149 [==============================] - 56s 374ms/step - loss: 1.3598 - acc: 0.6820 - val_loss: 1.0899 - val_acc: 0.7119
Epoch 3/10
148/149 [============================>.] - ETA: 0s - loss: 1.0747 - acc: 0.7331
Epoch 00003: acc improved from 0.68199 to 0.73398, saving model to d_model_rm.h5
149/149 [==============================] - 57s 379ms/step - loss: 1.0708 - acc: 0.7340 - val_loss: 0.9962 - val_acc: 0.7405
Epoch 4/10
148/149 [============================>.] - ETA: 0s - loss: 0.9064 - acc: 0.7631
Epoch 00004: acc improve